# Web scraping

## Co je web scraping?
- Strojové čtení nestrukturovaných dat z webových stránek

## Co není web scraping?
- Stahování dat přes API
- Stahování strukturovaných dat (JSON, CSV,...)
- Crawling - procházení a indexování celé webové stránky pomocí jejích vnitřních hypertextových odkazů


## Příklady web scrapingu

### Analýza cen na českých e-shopech


<div>
    <img src="static/hlidacshopu.png", width="50%"/>
</div>

https://medium.com/@jakubbalada/black-friday-2019-s-hl%C3%ADda%C4%8Dem-shop%C5%AF-9a3ddd352a8c

### Jak hodnotila filmy Mirka Spáčilová


<div>
<img src="static/spacilova.png",width="50%", style="width:400px;"/>
</div>

https://www.michalblaha.cz/2017/10/filmova-kriticka-mirka-spacilova-v-cislech/

## Etika web scrapingu

- Než začneš s web scrapingem, podívej se, jestli stránka nenabízí strukturovaná data ke stažení nebo neposkytuje API. 
    - **Příklady:** 
    - https://data.gov.cz/datov%C3%A9-sady?poskytovatel=%C4%8Cesk%C3%BD%20statistick%C3%BD%20%C3%BA%C5%99ad
    - https://www.ncdc.noaa.gov/data-access
    - https://www.mapakriminality.cz/data/
    - http://opendata.praha.eu/dataset/meteostanice-chmi-api
- Zjisti si, jaká máš práva k datům, nepublikuj získaná data nelegálně
- Přistupuj ke stránce k rozumné míře, nesnažíš se stránku shodit, ale získat data :-) 

## Z čeho se skládá webová stránka
- **HTML** (HyperText Markup Language): strukturovaný obsah stránky (text a obrázky)
- **CSS** (Cascading Style Sheets): úprava vzhledu stránky
- **JavaScript**: interaktivita obsahu a vzhledu stránky

### HTML 
<div>
    <img src="static/html.png", width="80%">
</div>

- Je tvořen HTML značkami / tagy, např.  ``<img>``
- Většina HTML tagů je párová, např. ``<h2>`` a ``</h2>``
- Tagy mohou mít atributy, které dále specifikují, co a jak bude tag zobrazovat
- Atribut class se obvykle používá k stylování stránky a často podle něj můžeme při webscrapingu odlišit různé části stránky

### CSS

- Popisuje způsob zobrazení html elementů
- Obsahuje 2 části: selektor elementu a blok deklarace:

```
p.error {
  color: red;
}
```
